In [1]:
import os
import pandas as pd
import sqlite3
import matplotlib.pyplot as plt

# Reading Data using Pandas and SQLite3

In [2]:
# helper function to join tables across different database (.db) files
def join_tables(tables):
    
    df = tables[0]
    for table in tables[1:]:
        df = pd.concat([df, table])
        
    return df.reset_index(drop=True)

# joining all database (.db) files of folder
def get_all_databases(folder):
    
    string_tables = []
    ull_tables = []
    files = os.listdir(folder)
    
    # iterating through all files of folder to append tables
    for file in files:
        path = os.path.join(folder, file)
        con = sqlite3.connect(path)
        string_tables.append(pd.read_sql_query('SELECT * FROM COUNTERS_STRING_TIME_DATA', con))
        ull_tables.append(pd.read_sql_query('SELECT * FROM COUNTERS_ULL_TIME_DATA', con))
        
    # concatenating tables into single DataFrames
    string_df = join_tables(string_tables)
    string_df.loc[:, 'VALUE'] = string_df.loc[:, 'VALUE'].replace({'WWAHost.exe': 'Netflix.exe'}) 
    ull_df = join_tables(ull_tables)
    
    return (string_df, ull_df)

In [3]:
# LAPTOP-MP2GILK8
string_df_0, ull_df_0 = get_all_databases('group3_collected_data')

# DESKTOP-E83M0TM
string_df_1, ull_df_1 = get_all_databases('group3_collected_data_pc_1')

In [4]:
display(string_df_0.head())
display(ull_df_0.head())

,MEASUREMENT_TIME,ID_INPUT,VALUE,PRIVATE_DATA
0,2021-11-20 21:30:29.491,2,IDC_ARROW,0
1,2021-11-20 21:30:30.578,3,cmd.exe,0
2,2021-11-20 21:30:31.512,2,IDC_SIZEWE,0
3,2021-11-20 21:30:31.847,2,IDC_ARROW,0
4,2021-11-20 21:30:34.994,2,IDC_HAND,0


,MEASUREMENT_TIME,ID_INPUT,VALUE,PRIVATE_DATA
0,2021-11-20 21:30:29.436,0,791,0
1,2021-11-20 21:30:29.436,1,108,0
2,2021-11-20 21:30:30.442,0,814,0
3,2021-11-20 21:30:30.442,1,70,0
4,2021-11-20 21:30:30.578,4,2072,0


# Exploratory Data Analysis

In [20]:
# getting value counts of foreground applications on LAPTOP-MP2GILK8
foreground_apps = string_df_0[string_df_0['ID_INPUT'] == 3]['VALUE']
foreground_apps = foreground_apps.tolist()

In [21]:
pairs = []
for i in range(len(foreground_apps)-1):
    pairs.append((foreground_apps[i], foreground_apps[i+1]))
pairs

[('cmd.exe', 'explorer.exe'),
 ('explorer.exe', 'chrome.exe'),
 ('chrome.exe', 'explorer.exe'),
 ('explorer.exe', 'chrome.exe'),
 ('chrome.exe', 'msedge.exe'),
 ('msedge.exe', 'chrome.exe'),
 ('chrome.exe', 'Spotify.exe'),
 ('Spotify.exe', 'ShellExperienceHost.exe'),
 ('ShellExperienceHost.exe', 'SystemSettings.exe'),
 ('SystemSettings.exe', 'Spotify.exe'),
 ('Spotify.exe', 'SystemSettings.exe'),
 ('SystemSettings.exe', 'chrome.exe'),
 ('chrome.exe', 'Spotify.exe'),
 ('Spotify.exe', 'chrome.exe'),
 ('chrome.exe', 'Spotify.exe'),
 ('Spotify.exe', 'explorer.exe'),
 ('explorer.exe', 'chrome.exe'),
 ('chrome.exe', 'Spotify.exe'),
 ('Spotify.exe', 'chrome.exe'),
 ('chrome.exe', 'Spotify.exe'),
 ('Spotify.exe', 'explorer.exe'),
 ('explorer.exe', 'cmd.exe'),
 ('cmd.exe', 'msedge.exe'),
 ('msedge.exe', 'explorer.exe'),
 ('explorer.exe', 'chrome.exe'),
 ('chrome.exe', 'explorer.exe'),
 ('explorer.exe', 'chrome.exe'),
 ('chrome.exe', 'msedge.exe'),
 ('msedge.exe', 'chrome.exe'),
 ('chrome.exe', 

In [ ]:
applist = pd.Series(foreground_apps).unique()
applist = applist.tolist()

In [32]:
l = {}
for i in applist:
    k = {}
    for j in pairs:
        if j[0] == i:
            if j[1] not in k:
                k[j[1]] = 1
            else:
                k[j[1]] += 1
    l[i] = k
l

{'cmd.exe': {'explorer.exe': 55,
  'msedge.exe': 3,
  'chrome.exe': 117,
  'Discord.exe': 15,
  'ShellExperienceHost.exe': 4,
  'SearchApp.exe': 3,
  'python.exe': 6,
  'Teams.exe': 2,
  'Spotify.exe': 9,
  'cmd.exe': 8,
  'notepad++.exe': 2,
  'devenv.exe': 7,
  'Microsoft.AAD.BrokerPlugin.exe': 1,
  'DB Browser for SQLite.exe': 2,
  'SystemSettings.exe': 1,
  'ScreenClippingHost.exe': 2,
  'vpnui.exe': 1,
  'Anaconda3-2021.11-Windows-x86_64.exe': 1},
 'explorer.exe': {'chrome.exe': 248,
  'cmd.exe': 111,
  'Discord.exe': 33,
  'Teams.exe': 57,
  'Zoom.exe': 15,
  'SystemSettings.exe': 20,
  'DB Browser for SQLite.exe': 84,
  'notepad++.exe': 4,
  'SearchApp.exe': 6,
  'Taskmgr.exe': 4,
  'mmc.exe': 2,
  'Spotify.exe': 18,
  'SndVol.exe': 16,
  'Microsoft.Notes.exe': 1,
  'GitHubDesktop.exe': 3,
  'notepad.exe': 3,
  'msedge.exe': 6,
  'devenv.exe': 59,
  'OpenWith.exe': 1,
  'VsDebugConsole.exe': 1,
  'HxOutlook.exe': 1,
  'ShellExperienceHost.exe': 4,
  'Microsoft.AAD.BrokerPlugin.e

In [35]:
sum(l['cmd.exe'].values())

239

In [36]:
for i in l.keys():
    s = sum(l[i].values())
    for j in l[i].keys():
        l[i][j] = l[i][j] / s
l

{'cmd.exe': {'explorer.exe': 0.2301255230125523,
  'msedge.exe': 0.012552301255230125,
  'chrome.exe': 0.4895397489539749,
  'Discord.exe': 0.06276150627615062,
  'ShellExperienceHost.exe': 0.016736401673640166,
  'SearchApp.exe': 0.012552301255230125,
  'python.exe': 0.02510460251046025,
  'Teams.exe': 0.008368200836820083,
  'Spotify.exe': 0.03765690376569038,
  'cmd.exe': 0.03347280334728033,
  'notepad++.exe': 0.008368200836820083,
  'devenv.exe': 0.029288702928870293,
  'Microsoft.AAD.BrokerPlugin.exe': 0.0041841004184100415,
  'DB Browser for SQLite.exe': 0.008368200836820083,
  'SystemSettings.exe': 0.0041841004184100415,
  'ScreenClippingHost.exe': 0.008368200836820083,
  'vpnui.exe': 0.0041841004184100415,
  'Anaconda3-2021.11-Windows-x86_64.exe': 0.0041841004184100415},
 'explorer.exe': {'chrome.exe': 0.33972602739726027,
  'cmd.exe': 0.15205479452054796,
  'Discord.exe': 0.045205479452054796,
  'Teams.exe': 0.07808219178082192,
  'Zoom.exe': 0.02054794520547945,
  'SystemSet